In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import json
import re
import string 
%matplotlib inline

In [2]:
# Load master data
path = "~/Documents/thesis/data/csv/FacebookAds.csv"
df = pd.read_csv(path)

## Scrub df of missing or unwanted values

In [3]:
# Remove rows with null values
remove_rows_for_cols = ['AdText','AdID','CreationDate']
for col in remove_rows_for_cols:
    df = df[pd.notnull(df[col])]

# Reset index after removing rows
df = df.reset_index(drop=True)

In [4]:
# Remove rows without valid text
for n, e in enumerate(df['AdText']):
    if e and re.match(r'^[_\W]+$', e):
        df.at[n, 'AdText'] = None

In [5]:
# Remove rows with null AdText values
for col in remove_rows_for_cols:
    df = df[pd.notnull(df['AdText'])]

# Reset index after removing rows
df = df.reset_index(drop=True)

In [6]:
# Scrub texts of social media phrases
social_media_phrases = ["Repost","Subscribe to our channel:","Follow my Facebook:","Follow me on Instagram:","Follow me on Twitter:"]
for n, e in enumerate(df['AdText']):
    curr_e = e.lower()
    curr_e = re.sub(r'http\S+', '', curr_e)
    [
        curr_e.replace(phrase, '') for phrase in social_media_phrases
    ]
    df.at[n, 'AdText'] = curr_e

In [7]:
df = df.drop(columns=[
    'EndDate', 'Behaviors', 'PeopleWhoMatch', 'Language','FriendsOfConnections', 
    'ExcludedConnections', 'Gender', 'Generation',
    'Politics', 'CustomAudience', 'SourceFile', 
    'SourceZip', 'pages', 'Location'
])
df = df.reset_index(drop=True)

In [8]:
# Fix integer colums with null vals
integer_cols = ['Clicks','Impressions','AdSpend']
for integer_col in integer_cols:
    df[integer_col] = df[integer_col].fillna(0.0)
    
# Fix string columns with null vals
string_cols = ["LandingPage", "Interests","AdSpendCurrency"]
for string_col in string_cols:
    df[string_col] = df[string_col].fillna("Unavailable")

In [9]:
# Count null values as percentage for each column
total_vals = len(df)
for col in df.columns:
    null_vals = df[col].isnull().sum()
    print("{0}: {1}".format(col, float(null_vals)/total_vals))

AdID: 0.0
AdText: 0.0
Clicks: 0.0
Impressions: 0.0
Age: 0.0
CreationDate: 0.0
LandingPage: 0.0
Placements: 0.0
Interests: 0.0
AdSpend: 0.0
AdSpendCurrency: 0.0


## Create new columns for better covariates

In [10]:
# CreationDateFormatted
CreationDateFormatted = []
for CreationDate in df['CreationDate']:
    try:
        new_date = dateutil.parser.parse(CreationDate[:-7]).date()
        CreationDateFormatted.append(new_date)
    except:
        CreationDateFormatted.append(dateutil.parser.parse('2015-10-31'))
df['CreationDateFormatted'] = CreationDateFormatted

In [11]:
# AgeAverage
AgeAverage = []
for age_string in df['Age']:
    l = re.findall(r'\d+', age_string)
    l = [float(n) for n in l]
    AgeAverage.append(sum(l) / float(len(l)))
df['AgeAverage'] = AgeAverage

In [12]:
# AgeAverageBin
df['AgeAverageBin'] = pd.qcut(df['AgeAverage'], 4, labels=["LowAge","MidAge","HighAge"],duplicates='drop')

In [13]:
# AdSpendBin
df['AdSpendBin'] = pd.qcut(df['AdSpend'], 3, labels=["low","mid","high"])

In [14]:
# ClicksBin
df['ClicksBin'] = pd.qcut(df['Clicks'], 3, labels=["low","mid","high"])

In [15]:
# ImpressionsBin
df["ImpressionsBin"] = pd.qcut(df['Impressions'], 3, labels=["low","mid","high"])

In [16]:
# InterestsGroups

# Load mapped interests grups json file
interests_path = "/Users/drewnleonard/Documents/thesis/data/json/interest_groups.json"
with open(interests_path) as f:
    interests_groups_map = json.load(f)

for k, v in interests_groups_map.iteritems():
    v = [e.lower() for e in v]
    interests_groups_map[k] = set(v)

interests_group_master = {}
    
for n, interests in enumerate(df['Interests']):
    
    # Continue if there are no available interests
    if interests == 'Unavailable':
        continue
    
    # Parse interests into list
    interests_list = interests.split(',')
    
    found_interest_groups = {}
    
    # Iterate over interests in list
    for interest in interests_list:
        
        interest = interest.lower()
        
        # Iterate over mapped groups
        for interest_group_title, interest_group_keywords in interests_groups_map.iteritems():
            
            # For each mapped group, iterate over its keywords
            for keyword in interest_group_keywords:
                    
                    # If keyword is in interest, record that and break
                    if keyword in interest:
                        
                        if interest_group_title not in found_interest_groups:
                            found_interest_groups[interest_group_title] = 0
                        
                        # Increment keyword's value 
                        found_interest_groups[interest_group_title] += 1
                        
                        break
    
    curr_group_title = "Unavailable"
    curr_group_score = 0
    
    for k, v in found_interest_groups.iteritems():
        curr_group_title = k if v > curr_group_score else curr_group_title
        
    df.at[n, 'Interests'] = curr_group_title
                               
# Algorithm:
# Convert interests_groups_map values to sets, lowercase
# For each ad
# If interests are "Unavailable", continue
# Split interests field into list
# For each individual interest
# Check whether that interest is part of any value set in interests_groups_map
# If so, tally that for given interest group
# At the end, pick interest group with highest score

In [20]:
df.head()

,AdID,AdText,Clicks,Impressions,Age,CreationDate,LandingPage,Placements,Interests,AdSpend,AdSpendCurrency,CreationDateFormatted,AgeAverage,AgeAverageBin,AdSpendBin,ClicksBin,ImpressionsBin
0,374,join us because we care. black matters.,0.0,137.0,18 - 65+,06/10/15 02:59:53 AM PDT,https://www.facebook.com/Black-Matters-1579673...,News Feed on desktop computers or News Feed on...,Unavailable,44.87,RUB,2015-06-10,41.5,MidAge,mid,low,mid
1,655,not every boy wants to be a soldier. a beautif...,35.0,452.0,18 - 65+,06/23/15 07:04:01 AM PDT,https://www.facebook.com/LGBT-United-839497472...,News Feed on desktop computers or News Feed on...,Unavailable,184.81,RUB,2015-06-23,41.5,MidAge,mid,mid,mid
2,664,"""people can tolerate two homosexuals they see ...",26.0,374.0,18 - 65+,06/23/15 07:02:40 AM PDT,https://www.facebook.com/LGBT-United-839497472...,News Feed on desktop computers or News Feed on...,Unavailable,99.95,RUB,2015-06-23,41.5,MidAge,mid,mid,mid
3,325,california... knows how to party california......,4.0,326.0,18 - 65+,06/10/15 07:34:52 AM PDT,https://www.facebook.com/Black-Matters-1579673...,News Feed on desktop computers or News Feed on...,Unavailable,45.94,RUB,2015-06-10,41.5,MidAge,mid,mid,mid
4,326,"since 2010, over 350 of our lives have been ta...",517.0,1478.0,18 - 65+,06/12/15 03:13:16 AM PDT,https://www.facebook.com/Black-Matters-1579673...,News Feed on desktop computers or News Feed on...,Unavailable,99.97,RUB,2015-06-12,41.5,MidAge,mid,high,mid


In [23]:
df.to_csv('~/Documents/thesis/data/csv/fb_gold.csv',index=False)